# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key
from api_keys import weather_api_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Save config information.
url = "http://api.openweathermap.org/data/2.5/weather?"
units = "imperial"

# Build partial query URL
query_url = f"{url}appid={weather_api_key}&units={units}&q="

cities_csv = "output_data/cities.csv"

cities_df = pd.read_csv(cities_csv)
cities_df

,City,Latitude,Longitude,Max Tem,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Country,ID,Date
0,busselton,-33.6500,115.3333,68.00,68.00,81,100,13.29,AU,2075265,1616077359
1,north bend,43.4065,-124.2243,44.60,43.38,93,90,5.75,US,5742974,1616077369
2,punta arenas,-53.1500,-70.9167,55.40,55.40,58,40,26.46,CL,3874787,1616077369
3,kodiak,57.7900,-152.4072,32.00,32.00,43,20,11.50,US,5866583,1616077370
4,port hedland,-20.3167,118.5667,80.60,80.60,57,0,17.27,AU,2063042,1616077370
...,...,...,...,...,...,...,...,...,...,...,...
569,chalmette,29.9427,-89.9634,60.01,57.83,54,1,11.50,US,4319518,1616077540
570,acapulco,16.8634,-99.8901,75.20,75.20,83,1,4.61,MX,3533462,1616077541
571,surt,31.2089,16.5887,65.50,65.50,52,100,14.32,LY,2210554,1616077541
572,malpura,26.2833,75.3833,81.46,81.46,25,76,20.85,IN,1264032,1616077541


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
humidity = cities_df['Humidity (%)'].astype(float)
maxhumidity = humidity.max()
locations = cities_df[['Latitude', 'Longitude']]
locations

,Latitude,Longitude
0,-33.6500,115.3333
1,43.4065,-124.2243
2,-53.1500,-70.9167
3,57.7900,-152.4072
4,-20.3167,118.5667
...,...,...
569,29.9427,-89.9634
570,16.8634,-99.8901
571,31.2089,16.5887
572,26.2833,75.3833


In [13]:
figure_layout = {'width': '700px', 'margin': '0 auto 0 auto'}
fig = gmaps.figure(layout = figure_layout)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,dissipating=False, max_intensity=maxhumidity,point_radius=3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px', margin='0 auto 0 auto', width='700px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
selected_city_df = cities_df.loc[(cities_df['Max Tem'] > 70) & (cities_df['Max Tem'] < 80) \
                                 & (cities_df['Cloudiness (%)'] == 0) & (cities_df['Wind Speed (mph)'] < 10), :]
selected_city_df = selected_city_df.dropna(how='any')
selected_city_df.reset_index(inplace=True)
del selected_city_df['index']
selected_city_df.head(20)
#print(len(selected_city_df))

#Lat = selected_city_df.loc[2]['Latitude']
#print(Lat)

,City,Latitude,Longitude,Max Tem,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Country,ID,Date
0,pisco,-13.7000,-76.2167,73.40,71.92,73,0,5.75,PE,3932145,1616077378
1,hermanus,-34.4187,19.2345,73.00,73.00,73,0,8.99,ZA,3366880,1616077380
2,bull savanna,17.8868,-77.5902,77.14,77.14,66,0,9.62,JM,3491161,1616077421
3,geraldton,-28.7667,114.6000,71.60,71.60,88,0,5.75,AU,2070998,1616077425
4,roxana,10.2753,-83.7513,78.80,76.14,100,0,6.91,CR,3622036,1616077437
5,jalu,29.0331,21.5482,73.26,73.26,23,0,7.02,LY,86049,1616077446
6,tura,25.5198,90.2201,73.20,73.20,21,0,3.20,IN,1254046,1616077454
7,hit,33.6416,42.8251,71.11,71.11,29,0,9.31,IQ,95788,1616077497
8,sao filipe,14.8961,-24.4956,73.31,73.31,72,0,2.84,CV,3374210,1616077518
9,san francisco menendez,13.8431,-90.0158,71.60,71.60,68,0,3.20,SV,3583716,1616077531


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_list = []
#Created a list of cities
for i in range(len(selected_city_df)):
    Lat = selected_city_df.loc[i]['Latitude']
    Lon = selected_city_df.loc[i]['Longitude']

    params = {
        'location': f'{Lat},{Lon}',
        'radius': 5000,
        'types' : 'hotel',
        'key': g_key
    }
    
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
    requested = requests.get(base_url, params=params)
    response = requested.json()
    #print(response['results'][0]['name'])
    try:
        hotel_list.append(response['results'][0]['name'])
    except:
        hotel_list.append('')
selected_city_df['Hotel Name'] = hotel_list
selected_city_df = selected_city_df.dropna(how='any')
selected_city_df.head(12)
print(hotel_list)

['Pisco', 'Hermanus', 'Alligator Pond', 'Geraldton', 'Guapiles', 'Jalu', 'Tura', 'Hit', 'Sao Filipe', 'San Francisco Menedez', 'Lima Duarte', 'Bara Mohansingh']


In [7]:
selected_city_df.head(12)

,City,Latitude,Longitude,Max Tem,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Country,ID,Date,Hotel Name
0,pisco,-13.7000,-76.2167,73.40,71.92,73,0,5.75,PE,3932145,1616077378,Pisco
1,hermanus,-34.4187,19.2345,73.00,73.00,73,0,8.99,ZA,3366880,1616077380,Hermanus
2,bull savanna,17.8868,-77.5902,77.14,77.14,66,0,9.62,JM,3491161,1616077421,Alligator Pond
3,geraldton,-28.7667,114.6000,71.60,71.60,88,0,5.75,AU,2070998,1616077425,Geraldton
4,roxana,10.2753,-83.7513,78.80,76.14,100,0,6.91,CR,3622036,1616077437,Guapiles
5,jalu,29.0331,21.5482,73.26,73.26,23,0,7.02,LY,86049,1616077446,Jalu
6,tura,25.5198,90.2201,73.20,73.20,21,0,3.20,IN,1254046,1616077454,Tura
7,hit,33.6416,42.8251,71.11,71.11,29,0,9.31,IQ,95788,1616077497,Hit
8,sao filipe,14.8961,-24.4956,73.31,73.31,72,0,2.84,CV,3374210,1616077518,Sao Filipe
9,san francisco menendez,13.8431,-90.0158,71.60,71.60,68,0,3.20,SV,3583716,1616077531,San Francisco Menedez


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in selected_city_df.iterrows()]
locations_h = selected_city_df[['Latitude', 'Longitude']]

In [14]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations_h)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px', margin='0 auto 0 auto', width='700px'))